In [1]:
# Running this code will query a table in BigQuery and download
# the results to a Pandas DataFrame named `results`.
# Learn more here: https://cloud.google.com/bigquery/docs/visualize-jupyter


%%bigquery results --project my-new-project-457009
SELECT * FROM `my-new-project-457009.GA_Sessions_Data.Funnel_Data_New` #this table name was set based on the table you chose to query

Query is running:   0%|          |

Downloading:   0%|          |

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# You can view the resulting Pandas DataFrame and work with using the Pandas library.
# https://pandas.pydata.org/docs/getting_started/index.html#getting-started
print(results.shape)
results.isna().sum()

(10939, 13)


,0
fullVisitorId,0
visitStartTime,0
hitNumber,0
time,0
isEntrance,8383
isExit,8383
pagePath,0
pageTitle,12
browser,0
isMobile,0


In [4]:
print(results[results['pageTitle'].isnull()].shape)
# results[results['pagePath']=='/storeitem.html']

(12, 13)


In [5]:
results['session_id'] = results['fullVisitorId'].astype(str) + '_' + results['visitStartTime'].astype(str)
results = results.sort_values(by=['session_id', 'hitNumber'])
results.head(3)

,fullVisitorId,visitStartTime,hitNumber,time,isEntrance,isExit,pagePath,pageTitle,browser,isMobile,deviceCategory,continent,country,session_id
1042,0004915997121163857,1501620292,1,0,True,True,/google+redesign/shop+by+brand/youtube,YouTube | Shop by Brand | Google Merchandise S...,Safari,True,mobile,Americas,United States,0004915997121163857_1501620292
4182,0013182148727187801,1501580378,1,0,True,<NA>,/home,Home,Safari,True,tablet,Europe,United Kingdom,0013182148727187801_1501580378
4183,0013182148727187801,1501580378,2,24780,<NA>,<NA>,/google+redesign/shop+by+brand/google,Google | Shop by Brand | Google Merchandise Store,Safari,True,tablet,Europe,United Kingdom,0013182148727187801_1501580378


In [6]:
session_paths = results.groupby(['session_id', 'browser', 'isMobile', 'deviceCategory','country'])['pagePath'].apply(list).reset_index()
session_paths.head(3)

,session_id,browser,isMobile,deviceCategory,country,pagePath
0,0004915997121163857_1501620292,Safari,True,mobile,United States,[/google+redesign/shop+by+brand/youtube]
1,0013182148727187801_1501580378,Safari,True,tablet,United Kingdom,"[/home, /google+redesign/shop+by+brand/google,..."
2,0015547161740087073_1501625964,Safari,True,tablet,United States,[/google+redesign/office/notebooks+journals/yo...


In [7]:
# results['pagePath'].value_counts().rename_axis('pagePath').reset_index(name='counts').to_excel('pagePathCounts.xlsx', index=False)
# from google.colab import files
# files.download('pagePathCounts.xlsx')


Approach 1: Mannual assingments.

In [8]:
# funnel_steps = {
#     'Landing Page': [
#         '/home', '/store.html', '/index.html', '/google+redesign/shop+by+brand/youtube',
#         '/search.html', '/asearch.html', '/quickview.html'
#     ],
#     'Product Page': [
#         '/google+redesign/apparel', '/google+redesign/apparel/mens',
#         '/google+redesign/apparel/womens', '/google+redesign/bags',
#         '/google+redesign/drinkware', '/google+redesign/electronics',
#         '/google+redesign/accessories', '/google+redesign/office',
#         '/google+redesign/apparel/headgear', '/google+redesign/apparel/kids',
#         '/google+redesign/accessories/fun', '/google+redesign/accessories/stickers/home',
#         '/google+redesign/apparel/mens/mens+t+shirts',
#         '/store.html/quickview',
#         '/google+redesign/bags/backpacks/home',
#         '/google+redesign/shop+by+brand/youtube/quickview',
#         '/google+redesign/office/notebooks++journals',
#         '/google+redesign/apparel/womens/womens+t+shirts',
#         '/google+redesign/apparel/mens/mens+outerwear',
#         '/google+redesign/bags/water+bottles+and+tumblers',
#         '/google+redesign/drinkware/mugs+and+cups',
#         '/google+redesign/shop+by+brand/google',
#         '/google+redesign/bags/quickview',
#         '/google+redesign/electronics/audio',
#         '/google+redesign/electronics/electronics+accessories'
#     ],
#     'Cart': [
#         '/basket.html'
#     ],
#     'Checkout': [
#         '/signin.html', '/yourinfo.html', '/payment.html',
#         '/checkout.html', '/address.html', '/shipping.html', '/revieworder.html'
#     ],
#     'Thank You': [
#         '/ordercompleted.html'
#     ]
# }

In [9]:
# def track_funnel_steps(path_list, funnel_def):
#     reached = {}
#     for step, keywords in funnel_def.items():
#         # Check if any path in the session matches the step's keywords
#         reached[step] = any(any(keyword in path for keyword in keywords) for path in path_list)
#     return reached

# funnel_flags = session_paths['pagePath'].apply(lambda path_list: track_funnel_steps(path_list, funnel_steps))
# funnel_df = pd.DataFrame(funnel_flags.tolist())

# funnel_df['session_id'] = session_paths['session_id']

Approach 2: Regex

In [10]:
def classify_page_path(path):
    if 'basket' in path or 'cart' in path:
        return 'Cart'
    elif 'checkout' in path or 'payment' in path or 'signin' in path or 'yourinfo' in path or 'revieworder' in path:
        return 'Checkout'
    elif 'ordercompleted' in path:
        return 'Thank You'
    elif any(x in path for x in ['apparel', 'bags', 'accessories', 'electronics', 'drinkware', 'notebooks']):
        return 'Product Page'
    elif any(x in path for x in ['home', 'store', 'search', 'quickview']):
        return 'Landing Page'
    else:
        return 'Other'
results['step_type'] = results['pagePath'].apply(classify_page_path)
results['step_type'].value_counts()

,count
step_type,
Product Page,4617
Landing Page,3192
Other,1585
Checkout,723
Cart,722
Thank You,100


In [11]:
funnel_df = results.groupby('session_id')['step_type'].apply(list).reset_index()

funnel_steps_list = ['Landing Page', 'Product Page', 'Cart', 'Checkout', 'Thank You']

for step in funnel_steps_list:
    funnel_df[step] = funnel_df['step_type'].apply(lambda steps: step in steps)
funnel_df

,session_id,step_type,Landing Page,Product Page,Cart,Checkout,Thank You
0,0004915997121163857_1501620292,[Other],False,False,False,False,False
1,0013182148727187801_1501580378,"[Landing Page, Other, Landing Page, Landing Pa...",True,True,False,False,False
2,0015547161740087073_1501625964,[Product Page],False,True,False,False,False
3,0017557475227585700_1501610006,[Landing Page],True,False,False,False,False
4,0018642067897203467_1501622515,"[Other, Landing Page]",True,False,False,False,False
...,...,...,...,...,...,...,...
2551,9984204982451798185_1501601690,"[Landing Page, Landing Page, Product Page, Pro...",True,True,False,False,False
2552,9985654296698195402_1501619433,"[Other, Product Page]",False,True,False,False,False
2553,9987807311191587328_1501601892,[Landing Page],True,False,False,False,False
2554,9990168985913952933_1501608605,"[Landing Page, Product Page, Product Page, Pro...",True,True,True,True,False


# Funnel Performance

In [12]:

funnel_counts = pd.DataFrame({
    'step': funnel_steps_list,
    'sessions': funnel_df[funnel_steps_list].sum().values
})
funnel_counts['next_step_sessions'] = funnel_counts['sessions'].shift(-1)

funnel_counts['conversion_rate_to_next'] = (
    funnel_counts['next_step_sessions'] / funnel_counts['sessions']
).apply(lambda x: f"{x:.1%}" if pd.notnull(x) else "—")

In [13]:
fig = go.Figure(go.Funnel(
    y=funnel_counts['step'],
    x=funnel_counts['sessions'],
    textinfo="value+percent previous",
    hovertext=[
        f"{step}<br>Sessions: {sessions}<br>Conversion to next: {rate}"
        for step, sessions, rate in zip(
            funnel_counts['step'],
            funnel_counts['sessions'],
            funnel_counts['conversion_rate_to_next']
        )
    ],
    hoverinfo="text"
))

fig.update_layout(title="User Funnel with Conversion Rates")
fig.show()


Looks like Cart → Checkout is flipped — maybe some users jump straight to checkout (will dig into that later).

In [14]:
checkout_only = funnel_df[
    (funnel_df['Checkout'] == True) &
    (funnel_df['Cart'] == False)
]

print(f"Sessions that skipped Cart and went to Checkout: {len(checkout_only)}")

Sessions that skipped Cart and went to Checkout: 114


Interestingly, 114 sessions went straight to checkout without visiting the cart page.

# Funnel Performance by Standard and Fast-Tracked Checkout Types

We can create two funnels: standard (Landing → Product → Cart → Checkout → Thank You) and fast-track (Landing/Product → Checkout → Thank You) for more clear picture.

In [15]:
funnel_df['skipped_cart'] = (funnel_df['Checkout'] == True) & (funnel_df['Cart'] == False)

# creating funnel types
funnel_df['funnel_type'] = 'standard'
funnel_df.loc[
    (funnel_df['Checkout'] == True) & (funnel_df['Cart'] == False),
    'funnel_type'
] = 'fast_track'


funnel_grouped = (
    funnel_df.groupby('funnel_type')[funnel_steps_list]
    .sum()
    .transpose()
    .reset_index()
    .rename(columns={'index': 'step'})
)
funnel_grouped

funnel_type,step,fast_track,standard
0,Landing Page,49,1574
1,Product Page,30,1086
2,Cart,0,252
3,Checkout,114,154
4,Thank You,0,51


In [16]:
# textinfo="value + percent previous" for non-zero division cases
fig = go.Figure()

fig.add_trace(go.Funnel(
    name='Standard Funnel',
    y=funnel_grouped['step'],
    x=funnel_grouped['standard'],
    textinfo="value"
))

fig.add_trace(go.Funnel(
    name='Fast-Track Funnel',
    y=funnel_grouped['step'],
    x=funnel_grouped['fast_track'],
    textinfo="value"
))

fig.update_layout(title="Standard vs. Fast-Track Funnel")
fig.show()


Merging sessions_path table with funnel_df table.

In [17]:
full_df = pd.merge(funnel_df, session_paths[['session_id', 'browser', 'isMobile', 'deviceCategory','country']],
         on='session_id', how='left')
full_df.head(3)

,session_id,step_type,Landing Page,Product Page,Cart,Checkout,Thank You,skipped_cart,funnel_type,browser,isMobile,deviceCategory,country
0,0004915997121163857_1501620292,[Other],False,False,False,False,False,False,standard,Safari,True,mobile,United States
1,0013182148727187801_1501580378,"[Landing Page, Other, Landing Page, Landing Pa...",True,True,False,False,False,False,standard,Safari,True,tablet,United Kingdom
2,0015547161740087073_1501625964,[Product Page],False,True,False,False,False,False,standard,Safari,True,tablet,United States


# Funnel Performance by Device Type

In [18]:
device_funnel = (
    full_df.groupby('deviceCategory')[['Landing Page', 'Product Page', 'Cart', 'Checkout', 'Thank You']]
    .sum()
    .transpose()
    .reset_index()
    .rename(columns={'index': 'step'})
)
device_funnel

deviceCategory,step,desktop,mobile,tablet
0,Landing Page,1176,397,50
1,Product Page,804,280,32
2,Cart,207,41,4
3,Checkout,225,39,4
4,Thank You,49,2,0


In [19]:
fig = go.Figure()

for col in device_funnel.columns[1:]:
    fig.add_trace(go.Funnel(
        name=col,
        y=device_funnel['step'],
        x=device_funnel[col],
        textinfo="value+percent previous"
    ))

fig.update_layout(title="Funnel Comparison by Device Type")
fig.show()




*   The desktop funnel is healthy and converting at ~4.2% (49 / 1176)

* Mobile's conversion rate is just 0.5%

* Tablet has a 0% conversion rate

In [20]:
device_conv = full_df.groupby('deviceCategory')[['Checkout', 'Thank You']].sum()
device_conv['conversion_rate'] = (device_conv['Thank You'] / device_conv['Checkout']).apply(lambda x: f"{x:.1%}")
device_conv.reset_index()

,deviceCategory,Checkout,Thank You,conversion_rate
0,desktop,225,49,21.8%
1,mobile,39,2,5.1%
2,tablet,4,0,0.0%


Desktop users converted at 21.8%, compared to 5.1% on mobile. This suggests a need to improve mobile checkout UX.

# Drop_Off Points by Device Types

In [21]:
def calculate_dropoffs(funnel_df, column):
    previous = funnel_df[column].shift(1)
    current = funnel_df[column]
    dropoffs = previous - current
    dropoff_rate = (dropoffs / previous).apply(lambda x: f"{x:.1%}" if pd.notnull(x) else "—")
    return dropoffs.fillna(0).astype(int), dropoff_rate

# adding columns for each device
for device in ['desktop', 'mobile', 'tablet']:
    device_funnel[f'{device}_dropoff'], device_funnel[f'{device}_dropoff_rate'] = calculate_dropoffs(device_funnel, device)
device_funnel[['step', 'mobile_dropoff_rate', 'desktop_dropoff_rate', 'tablet_dropoff_rate']]

deviceCategory,step,mobile_dropoff_rate,desktop_dropoff_rate,tablet_dropoff_rate
0,Landing Page,—,—,—
1,Product Page,29.5%,31.6%,36.0%
2,Cart,85.4%,74.3%,87.5%
3,Checkout,4.9%,-8.7%,0.0%
4,Thank You,94.9%,78.2%,100.0%


There are 3 device types: desktop, mobile and tablet. All devices show major drop-off at the Add to Cart step, with mobile suffering the most post-checkout, which could be related to form usability, payment method availability, or page errors.

Suggestions: mobile-specific testing.

In [22]:
country_funnel = full_df.groupby('country')[['Landing Page', 'Product Page', 'Cart', 'Checkout', 'Thank You']].sum()

dropoff_data = []

steps = ['Landing Page', 'Product Page', 'Cart', 'Checkout', 'Thank You']


for i in range(1, len(steps)):
    prev_step = steps[i - 1]
    current_step = steps[i]

    # calculating drop-off count and rate
    dropoff_count = country_funnel[prev_step] - country_funnel[current_step]
    dropoff_rate = (dropoff_count / country_funnel[prev_step]).replace([float('inf'), -float('inf')], pd.NA)

    for country in country_funnel.index:
        dropoff_data.append({
            'country': country,
            'step': f'{prev_step} → {current_step}',
            'dropoff_count': int(dropoff_count[country]),
            'dropoff_rate': f"{dropoff_rate[country]:.1%}" if pd.notnull(dropoff_rate[country]) else '—'
        })

country_dropoffs = pd.DataFrame(dropoff_data)
country_dropoffs.sort_values(by='dropoff_count', ascending=False).head(10)

,country,step,dropoff_count,dropoff_rate
185,United States,Product Page → Cart,530,74.9%
373,United States,Checkout → Thank You,156,75.7%
91,United States,Landing Page → Product Page,141,16.6%
90,United Kingdom,Landing Page → Product Page,60,65.2%
30,Germany,Landing Page → Product Page,55,74.3%
131,India,Product Page → Cart,50,84.7%
37,India,Landing Page → Product Page,36,37.9%
107,Canada,Product Page → Cart,34,81.0%
184,United Kingdom,Product Page → Cart,27,84.4%
43,Japan,Landing Page → Product Page,17,58.6%


The US is the largest source of drop-offs with 530 users between Product Page and Cart and 156 users at Checkout → Thank You. India, UK and Germany are the second largest sources of drop-offs.

In [23]:
high_volume = country_funnel[country_funnel['Landing Page'] > 50].index
filtered_dropoffs = country_dropoffs[country_dropoffs['country'].isin(high_volume)]
filtered_dropoffs.sort_values(by='dropoff_rate', ascending=False).head(10)

,country,step,dropoff_count,dropoff_rate
124,Germany,Product Page → Cart,17,89.5%
131,India,Product Page → Cart,50,84.7%
184,United Kingdom,Product Page → Cart,27,84.4%
107,Canada,Product Page → Cart,34,81.0%
373,United States,Checkout → Thank You,156,75.7%
185,United States,Product Page → Cart,530,74.9%
30,Germany,Landing Page → Product Page,55,74.3%
90,United Kingdom,Landing Page → Product Page,60,65.2%
218,Germany,Cart → Checkout,1,50.0%
201,Canada,Cart → Checkout,4,50.0%


Across high-volume countries, the largest drop-offs occur at the Product Page → Cart step, especially in Germany (89.5%), India (84.7%), and the UK (84.4%).

In [24]:
top_dropoffs = country_dropoffs.sort_values(by='dropoff_count', ascending=False).head(10)
fig = px.bar(
    top_dropoffs,
    x='dropoff_count',
    y='step',
    color='country',
    text='dropoff_rate',
    title='Top Drop-Off Points by Country (High-Volume Only)',
    labels={'dropoff_count': 'Drop-Off Count', 'step': 'Funnel Step'})

fig.update_traces(textposition='outside')
fig.update_layout(yaxis=dict(categoryorder='total ascending'))

fig.show()

# Funnel Performance by Browser

In [25]:
browser_funnel = (
    full_df.groupby('browser')[['Landing Page', 'Product Page', 'Cart', 'Checkout', 'Thank You']]
    .sum()
    .transpose()
    .reset_index()
    .rename(columns={'index': 'step'})
)

browser_funnel['Android'] = browser_funnel['Android Browser'] + browser_funnel['Android Webview']
browser_funnel['Opera'] = browser_funnel['Opera'] + browser_funnel['Opera Mini']
browser_funnel['Safari'] = browser_funnel['Safari'] + browser_funnel['Safari (in-app)']
browser_funnel['Other'] = browser_funnel['Coc Coc']+ browser_funnel['Mozilla Compatible Agent'] + browser_funnel['Nokia Browser'] + browser_funnel['UC Browser'] + browser_funnel['YaBrowser']
browser_funnel = browser_funnel[['step', 'Android', 'Chrome', 'Edge', 'Firefox', 'Internet Explorer', 'Opera',
                                 'Safari', 'Other']]
browser_funnel

browser,step,Android,Chrome,Edge,Firefox,Internet Explorer,Opera,Safari,Other
0,Landing Page,13,1281,12,59,25,10,216,7
1,Product Page,5,875,8,33,15,13,165,2
2,Cart,2,221,2,4,2,0,20,1
3,Checkout,5,238,1,6,2,0,16,0
4,Thank You,0,47,0,1,0,0,3,0


In [26]:
dropoff_results = []
browsers = browser_funnel.columns[1:]

for browser in browsers:
    for i in range(1, len(steps)):
        prev = browser_funnel.iloc[i - 1][browser]
        curr = browser_funnel.iloc[i][browser]
        dropoff = prev - curr
        drop_rate = dropoff / prev if prev != 0 else None

        dropoff_results.append({
            'browser': browser,
            'step': f"{steps[i - 1]} → {steps[i]}",
            'dropoff_count': dropoff,
            'dropoff_rate': f"{drop_rate:.1%}" if drop_rate is not None else '—'
        })

dropoff_df = pd.DataFrame(dropoff_results)
dropoff_df = dropoff_df.sort_values(by=['dropoff_count',	'dropoff_rate'], ascending=False)
dropoff_df

,browser,step,dropoff_count,dropoff_rate
5,Chrome,Product Page → Cart,654,74.7%
4,Chrome,Landing Page → Product Page,406,31.7%
7,Chrome,Checkout → Thank You,191,80.3%
25,Safari,Product Page → Cart,145,87.9%
24,Safari,Landing Page → Product Page,51,23.6%
13,Firefox,Product Page → Cart,29,87.9%
12,Firefox,Landing Page → Product Page,26,44.1%
17,Internet Explorer,Product Page → Cart,13,86.7%
27,Safari,Checkout → Thank You,13,81.2%
21,Opera,Product Page → Cart,13,100.0%


As shown in the table, there are a few negative drop-off counts/rates. This means that more sessions were recorded in the next step than the previous step, likely due to stitching issues, missing hits in earlier steps, etc. Small counts (with 1 as value) are misleading, so I'll filter them out for a fair picture.

In [29]:
# setting the 'step' column as index so we can locate the Landing Page row easily
df_sessions = browser_funnel.set_index('step')

# using only the 'Landing Page' row to get initial session counts per browser
total_sessions = df_sessions.loc['Landing Page']
volume_flag = total_sessions.apply(lambda x: 'High' if x > 20 else 'Low')
dropoff_df['volume_flag'] = dropoff_df['browser'].map(volume_flag)
dropoff_df

,browser,step,dropoff_count,dropoff_rate,volume_flag
5,Chrome,Product Page → Cart,654,74.7%,High
4,Chrome,Landing Page → Product Page,406,31.7%,High
7,Chrome,Checkout → Thank You,191,80.3%,High
25,Safari,Product Page → Cart,145,87.9%,High
24,Safari,Landing Page → Product Page,51,23.6%,High
13,Firefox,Product Page → Cart,29,87.9%,High
12,Firefox,Landing Page → Product Page,26,44.1%,High
17,Internet Explorer,Product Page → Cart,13,86.7%,High
27,Safari,Checkout → Thank You,13,81.2%,High
21,Opera,Product Page → Cart,13,100.0%,Low


In [30]:
filtered = dropoff_df[dropoff_df['volume_flag'] == 'High']

fig = px.bar(
    filtered,
    x='dropoff_count',
    y='step',
    color='browser',
    text='dropoff_rate',
    title='Top Drop-Off Points by Broswer',
    labels={'dropoff_count': 'Drop-Off Count', 'step': 'Funnel Step'}, barmode='group')

fig.update_traces(textposition='outside')
fig.update_layout(yaxis=dict(categoryorder='total ascending'))

fig.show()

* Chrome: has a decent volume, but suffers at Cart & Checkout.
* Safari & Firefox: both have major drop-offs at Product to Cart and Checkout stages.
* Internet Explorer: has 100% abandonment at checkout.

Negative drop-offs at Cart-Checkout typically mean a few things:
* Users re-enter sessions mid-funnel (multi-tab),
* Cart step not tracked properly
* Checkout accessed via quick-buy links or bookmarks

Suggestion:
* Review tag firing for Cart pages
* Check session stitching in GA